In [1]:
#1: Install Required Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
# 2: Data Preparation
# Path to your dataset
data_dir = r"C:\Users\havar\Downloads\Project\augmented_images"
# Dimensions of input images
img_width, img_height = 224, 224  

# Parameters for training
batch_size = 55  # batch size
epochs = 30  
num_classes = 11
validation_split = 0.2

# Use ImageDataGenerator for rescaling only
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=validation_split
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Convert generators to datasets and repeat them
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
).repeat()

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
).repeat()


Found 12320 images belonging to 11 classes.
Found 3080 images belonging to 11 classes.


In [3]:
# 3: Load the Xception model without the top (classification) layer
base_model = Xception(weights='imagenet', include_top=False)

# Freeze the base model
base_model.trainable = False

# Add a global average pooling layer and a dense output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 23s 0us/step


In [4]:
# 4: Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_dataset,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/30
 76/224 ━━━━━━━━━━━━━━━━━━━━ 29:45 12s/step - accuracy: 0.5391 - loss: 1.5665

KeyboardInterrupt: 

In [ ]:
# Save the final model
output_folder = r"C:\Users\havar\Downloads\Project\DeepLearning_model_output\Trained Models\Xception"
os.makedirs(output_folder, exist_ok=True)
model.save(os.path.join(output_folder, "Xception_model.keras"))


In [ ]:
# 5: Evaluate and Save the model
_, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Evaluate the model on validation data
evaluation = model.evaluate(validation_generator)
print(f"Evaluation Loss: {evaluation[0]}")
print(f"Evaluation Accuracy: {evaluation[1]}")


In [ ]:
#6: Generate confusion matrix
# Predict classes for validation set
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
true_classes = validation_generator.classes

# Generate confusion matrix
conf_matrix = confusion_matrix(true_classes, y_pred_classes)

print("Confusion Matrix of Xception Model:")
print(conf_matrix)

# Generate classification report
class_names = list(validation_generator.class_indices.keys())
report = classification_report(true_classes, y_pred_classes, target_names=class_names)
print("\nClassification Report of Xception Model:")
print(report)


In [ ]:
# Create a DataFrame for the confusion matrix
df_cm = pd.DataFrame(conf_matrix, index=class_names, columns=class_names)

# Plotly heatmap with confusion matrix values
fig_cm = px.imshow(df_cm, 
                   labels=dict(x="Predicted", y="Actual", color="Count"),
                   x=class_names,
                   y=class_names,
                   color_continuous_scale='Blues',
                   text_auto=True)  # text_auto=True to display values

fig_cm.update_layout(
    title='Confusion Matrix For Xception Model',
    title_x=0.5, 
    xaxis_title='Predicted Classes',
    yaxis_title='Actual Classes', 
    autosize=False,
    width=1000,  
    height=800 )
fig_cm.show()


In [ ]:
# Generate classification report
class_names = list(validation_generator.class_indices.keys())
report = classification_report(true_classes, y_pred_classes, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Plot the classification report with Plotly
report_df = report_df.iloc[:-1, :-1]  # Exclude support and accuracy rows
fig = px.imshow(report_df.T, 
                text_auto=True, 
                aspect="auto", 
                color_continuous_scale='viridis')
fig.update_layout(
    title='Classification Report of Xception Model',
    title_x=0.5, 
    xaxis_title='Classes',
    yaxis_title='Metrics', 
    autosize=False,
    width=1100,  
    height=400  
)


In [ ]:
# 7: Plot training & validation accuracy and loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Xception Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Xception Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

output_path = os.path.join(output_folder, "output_plot")
os.makedirs(output_path, exist_ok=True)
plt.savefig(os.path.join(output_path,"Xception.png"))

plt.tight_layout()
plt.show()


In [ ]:
# 7: Plot training & validation accuracy and loss with Plotly
history_df = pd.DataFrame(history.history)

fig_acc = px.line(history_df, y=['accuracy', 'val_accuracy'], 
                  labels={'index': 'Epoch', 'value': 'Accuracy'}, 
                  title='Xception Model Accuracy')
fig_acc.update_layout(yaxis=dict(range=[0, 1]))
fig_acc.show()

fig_loss = px.line(history_df, y=['loss', 'val_loss'], 
                   labels={'index': 'Epoch', 'value': 'Loss'}, 
                   title='Xception Model Loss' )
fig_loss.show()
